In [1]:
!nvidia-smi


Mon Dec 22 20:43:39 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   49C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import torch
print("torch:", torch.__version__)
print("cuda_available:", torch.cuda.is_available())
print("torch.version.cuda:", torch.version.cuda)
print("device_name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else None)


torch: 2.9.0+cu126
cuda_available: True
torch.version.cuda: 12.6
device_name: Tesla T4


In [3]:
%%bash
set -e
rm -rf /content/yelp-prediction
git clone -b finetune_clip https://github.com/ascandone/yelp-prediction.git /content/yelp-prediction
cd /content/yelp-prediction




Cloning into '/content/yelp-prediction'...
Filtering content: 100% (70/70), 203.69 MiB | 21.87 MiB/s, done.


In [4]:
!pip install poetry==1.8.3


INFO: pip is looking at multiple versions of poetry-plugin-export to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.9/249.9 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.5/309.5 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.4/519.4 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.8/453.8 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 98.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 102.5 MB/s eta 0:00:00
  Attempting uninstall: keyring
    Found existing installation: keyring 25.7.0
    Uninstalling keyring-25.7.0:
      Successfully uninstalled keyring-25.7.0


In [5]:
!poetry config virtualenvs.create false


In [6]:
!nvidia-smi
!python -c "import torch; print('cuda_available:', torch.cuda.is_available()); print('device:', torch.cuda.get_device_name(0) if torch.cuda.is_available() else None)"


Mon Dec 22 20:44:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8             10W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [8]:
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [9]:
%%bash
set -e
cd /content/yelp-prediction
curl -sL https://raw.githubusercontent.com/ascandone/yelp-prediction/refs/heads/main/download-data.sh | bash


Archive:  ./temp/json.zip
   creating: ./temp/json/Yelp JSON/
  inflating: ./temp/json/Yelp JSON/Yelp Dataset Documentation & ToS copy.pdf  
  inflating: ./temp/json/__MACOSX/Yelp JSON/._Yelp Dataset Documentation & ToS copy.pdf  
  inflating: ./temp/json/Yelp JSON/yelp_dataset.tar  
  inflating: ./temp/json/__MACOSX/Yelp JSON/._yelp_dataset.tar  
Dataset_User_Agreement.pdf
yelp_academic_dataset_business.json
yelp_academic_dataset_checkin.json
yelp_academic_dataset_review.json
yelp_academic_dataset_tip.json
yelp_academic_dataset_user.json
Archive:  ./temp/photos.zip
   creating: ./temp/photos/Yelp Photos/
  inflating: ./temp/photos/Yelp Photos/yelp_photos.tar  
  inflating: ./temp/photos/__MACOSX/Yelp Photos/._yelp_photos.tar  
  inflating: ./temp/photos/Yelp Photos/Yelp Dataset Documentation & ToS.pdf  
  inflating: ./temp/photos/__MACOSX/Yelp Photos/._Yelp Dataset Documentation & ToS.pdf  


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4144M  100 4144M    0     0  55.2M      0  0:01:14  0:01:14 --:--:-- 55.4M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7102M  100 7102M    0     0  29.1M      0  0:04:03  0:04:03 --:--:-- 24.4M


In [10]:
%cd /content/yelp-prediction
!ls -lah
!ls -lah pyproject.toml


/content/yelp-prediction
total 1.2M
drwxr-xr-x 6 root root 4.0K Dec 22 20:43 .
drwxr-xr-x 1 root root 4.0K Dec 22 20:47 ..
drwxr-xr-x 5 root root 4.0K Dec 22 20:58 data
-rw-r--r-- 1 root root  852 Dec 22 20:43 download-data.sh
drwxr-xr-x 3 root root 4.0K Dec 22 20:43 frontend
drwxr-xr-x 9 root root 4.0K Dec 22 20:43 .git
-rw-r--r-- 1 root root  196 Dec 22 20:43 .gitattributes
-rw-r--r-- 1 root root   70 Dec 22 20:43 .gitignore
-rw-r--r-- 1 root root 226K Dec 22 20:43 poetry.lock
-rw-r--r-- 1 root root 696K Dec 22 20:43 predictions-analysis.ipynb
-rw-r--r-- 1 root root  736 Dec 22 20:43 pyproject.toml
-rw-r--r-- 1 root root 163K Dec 22 20:43 raw_dataset_analysis.ipynb
-rw-r--r-- 1 root root  217 Dec 22 20:43 README.md
drwxr-xr-x 3 root root 4.0K Dec 22 20:43 src
-rw-r--r-- 1 root root 736 Dec 22 20:43 pyproject.toml


In [11]:
%cd /content
!git clone https://github.com/ascandone/yelp-prediction.git
%cd /content/yelp-prediction
!git checkout finetune_clip
!ls -lah pyproject.toml


/content
fatal: destination path 'yelp-prediction' already exists and is not an empty directory.
/content/yelp-prediction
Already on 'finetune_clip'
Your branch is up to date with 'origin/finetune_clip'.
-rw-r--r-- 1 root root 736 Dec 22 20:43 pyproject.toml


In [ ]:
!poetry run python -u src/yelp_prediction/finetune_clip_last2.py \
  --out-dir "/content/drive/MyDrive/yelp_outputs/finetune_clip_last2" \
  --device cuda --amp \
  --epochs 7 \
  --batch-size 32 \
  --num-workers 2 \
  --split-seed 42 \
  --head mlp --mlp-dims 1024 512 --dropout 0.3 --act gelu --bounded \
  --loss mse --optimizer adamw --lr-head 0.0005 --lr-backbone 0.000005 --weight-decay 0.01 \
  --patience 5 \
  --select-best mae \
  --run-tag final_last2_5ppb_bs32


Skipping virtualenv creation, as specified in config file.
2025-12-22 20:58:20.081515: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766437100.340153    4814 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766437100.407804    4814 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766437100.933603    4814 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766437100.933643    4814 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766437100.93364